In [2]:
# imports
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# netList = []
# df = df[df['edges'] > 1]
# for x in df.folder_name_x.unique():
#     cell_df = df[df['folder_name_x'] == x]
#     for cc in cell_df.cc_x.unique(): 
#         network_df = cell_df[cell_df['cc_x'] == cc]
#         if len(netList) < 2: 
#             netList.append(network_df)

def plot_3D(df, df_dir): 
   for name_folder in df['folder_name_x'].unique():
      sidedf = df[df['folder_name_x'] == name_folder]

      element_df = sidedf[['folder_name_x', 'cc_x', 'line_id','element_length_(um)', 'element_average_width','element_pixel_intensity_488', 'element_pixel_intensity_405',
         'element_pixel_intensity_ratio', 'folder_name_x','element_Volume_Voxel', 'x_x', 'y_x', 'z_x']]   

      cc_df = sidedf[['folder_name_x','cc_x', 'x_x', 'y_x', 'z_x', 'line_id', 'cc_length_(um)', 'cc_vol_from_img_(um3)', 'branches', 'nodes', 'edges', 'cc_pixel_intensity_ratio','cc_pixel_intensity_405', 'cc_pixel_intensity_488', 'cc_average_degree_excludeFreeEnds', 'diameter']]
      node_df = sidedf[['folder_name_x', 'cc_x', 'line_id', 'node', 'degree', 'x_y', 'y_y', 'z_y']].dropna()

      merged = cc_df

      cclist = []

      xlist = []
      ylist = []
      zlist = []
      tlist = []

      nxlist = []
      nylist = []
      nzlist = []
      nclist = []

      num_of_cc = len(merged['cc_x'].unique())
      # remove nan values from getting the max

      # first get the values we are to plot 
      maxt = max(merged['cc_pixel_intensity_ratio'].dropna())
      for i in range(0, num_of_cc):
         merged_cc= merged[merged['cc_x'] == i]
         # repeat this for all the other CCs, and then plot them all on the same graph
         line_ids_in_cc = merged_cc['line_id'].unique()
         line_list = []
         for j in range(len(line_ids_in_cc)):
            merged_line = merged_cc[merged_cc['line_id'] == line_ids_in_cc[j]]
            # nodes_temp = merged_line[merged_line['Node'] > 0]

            x = merged_line['x_x'].to_numpy()
            y = merged_line['y_x'].to_numpy()
            z = merged_line['z_x'].to_numpy()
            t = merged_line['cc_pixel_intensity_ratio'].to_numpy()

            xlist.append(x)
            ylist.append(y)
            zlist.append(z)
            tlist.append(t)
            line_list.append([x, y, z, t])

         cclist.append([line_list, i])


      fig = plt.figure(figsize=(15, 15))
      ax = fig.add_subplot(111, projection='3d')
      # change colormap of ax

      # plot the values 

      for i in range(len(cclist)):
         line_list = cclist[i][0]
         cc_number = cclist[i][1]
         # generate a random color 
         color = np.random.rand(3,)
         addedCC = False
         for j in range(len(line_list)):
            x = line_list[j][0]
            y = line_list[j][1]
            z = line_list[j][2]
            t = line_list[j][3]

            N_points = len(x)

            t /= maxt
            for k in range(1, N_points):
                  ax.plot(x[k-1:k+1], y[k-1:k+1], z[k-1:k+1], c=color, linewidth=3, alpha=0.5)

                  # Add CC number as text next to the plotted point
                  if addedCC == False: 
                     ax.text(x[k], y[k], z[k], str(cc_number), color=color, fontsize=8)
                     addedCC = True

      # plot the nodes, where the color of the node is determined by the degree of the node
      # first get the max degree
      # create a list of 10 colors 
      # suggest another color

      colors = ['white', 'grey', 'black', 'yellow', 'cyan', 'orange', 'pink', 'magenta', 'black', 'purple']
      # now based on the degree, assign a color to the node, based on the color list
      # drop duplicate node_df values
      node_df = node_df.drop_duplicates(subset=['node'])
      nx = node_df['x_y'].to_numpy()
      ny = node_df['y_y'].to_numpy()
      nz = node_df['z_y'].to_numpy()
      deg = node_df['degree'].to_numpy()
      node_name = node_df['node'].to_numpy()
      

      for i in range(len(nx)):
         color_index = int(deg[i]) % 10
         node_number = int(node_name[i])
         ax.scatter(nx[i], ny[i], nz[i], color=colors[color_index], s=6)
         # ax.text(nx[i], ny[i], nz[i], str(node_number), color='black', fontsize=10)

      # Create a legend
      for i, color in enumerate(colors):
         ax.scatter([], [], color=color, s=30, label=f'PK {i}')
      
      ax.legend(scatterpoints=1, frameon=False, labelspacing=1, title='Degree', loc='upper right')

      ax.view_init(90, 270)
         # make the plot a bit longer
         # add a color bar
      ax.set_aspect('equal')
      # include a legend for the nodes, where each color is the degree of the node
      # do it so legend isl ike this red: 1, blue :2 etc
            
      # # plt.show()
      # plt.title(merged['folder_name_x'][0])

      # plt.savefig(os.path.join(os.path.dirname(df_dir),  name_folder) + '_node_3D.png', dpi=300)

   return xlist, ylist, zlist, tlist


In [3]:
# import the dataframe 
df = pd.read_csv('full_Table.csv')
# import the full_table_node file
df_node = pd.read_csv('full_Table_Node.csv')

# select any cell ful
cell_df = df[df['folder_name_x'] == df['folder_name_x'].values[0]]


### Finding centroid of edges, network, and the distance to those points, and order based on that

In [23]:
# for each network in the cell find the center and store it in the df as cc_center_x, cc_center_y, cc_center_z

for cc in cell_df.cc_x.unique(): 
    a = cell_df[cell_df['cc_x'] == cc]
    # get the center of the network
    net_minx, net_maxx  = a['x_x'].min(), a['x_x'].max()
    net_miny, net_maxy  = a['y_x'].min(), a['y_x'].max()
    net_minz, net_maxz  = a['z_x'].min(), a['z_x'].max()
    center_network = np.array([net_minx + (net_maxx - net_minx)/2, net_miny + (net_maxy - net_miny)/2, net_minz + (net_maxz - net_minz)/2])
    cell_df.loc[cell_df['cc_x'] == cc, ['cc_center_x', 'cc_center_y', 'cc_center_z']] = center_network

    for line_id in a['line_id'].unique():
        line_df = a[a['line_id'] == line_id]
        minx,  maxx  = line_df['x_x'].min(), line_df['x_x'].max()
        miny, maxy  = line_df['y_x'].min(), line_df['y_x'].max()
        minz, maxz  = line_df['z_x'].min(), line_df['z_x'].max()

        center_line_id = np.array([minx + (maxx - minx)/2, miny + (maxy - miny)/2, minz + (maxz - minz)/2])
        cell_df.loc[cell_df['line_id'] == line_id, ['center_x', 'center_y', 'center_z']] = center_line_id

        # find the distance of the center_line_id to the center_network
        distance = np.linalg.norm(center_line_id - center_network) # this is the same as a euclidian distance
        cell_df.loc[cell_df['line_id'] == line_id, ['distance_to_center']] = distance    
    
    a = a.drop_duplicates(subset=['line_id'])  
    a = a.sort_values(by=['distance_to_center'])
    z = zip(a['line_id'], a['distance_to_center'])
    for i, v in enumerate(z): 
        cell_df.loc[cell_df['line_id'] == v[0], ['order']] = i

### Finding the angles between the different line_id's in a network

In [22]:
a = cell_df[cell_df['cc_x'] == 42].drop_duplicates(subset=['line_id'])  
a = a.sort_values(by=['distance_to_center'])
z = zip(a['line_id'], a['distance_to_center'])
for i, v in enumerate(z): 
    cell_df.loc[cell_df['line_id'] == v[0], ['order']] = i

In [18]:
a

,Unnamed: 0,cc_x,cc_length_(um),cc_vol_from_img_(um3),branches,nodes,edges,cc_pixel_intensity_ratio,cc_pixel_intensity_488,cc_pixel_intensity_405,...,folder_name_y,coeff_of_variance_cc,coeff_of_variance_line_id,cc_center_x,cc_center_y,cc_center_z,center_x,center_y,center_z,distance_to_center
1377,1377,42,2.94521,0.86528,3,4,3,0.062708,699.675,43.875,...,TCDD1nM4hr1_Subset,16.430575,0.000000e+00,10.09003,28.6,2.25,9.897335,28.045335,2.166665,0.593068
1378,1378,42,2.94521,0.86528,3,4,3,0.062708,699.675,43.875,...,NaN,16.430575,0.000000e+00,10.09003,28.6,2.25,9.897335,28.045335,2.166665,0.593068
1379,1379,42,2.94521,0.86528,3,4,3,0.062708,699.675,43.875,...,NaN,16.430575,0.000000e+00,10.09003,28.6,2.25,9.897335,28.045335,2.166665,0.593068
1380,1380,42,2.94521,0.86528,3,4,3,0.062708,699.675,43.875,...,TCDD1nM4hr1_Subset,16.430575,0.000000e+00,10.09003,28.6,2.25,9.795365,28.669335,1.916665,0.450274
1381,1381,42,2.94521,0.86528,3,4,3,0.062708,699.675,43.875,...,NaN,16.430575,0.000000e+00,10.09003,28.6,2.25,9.795365,28.669335,1.916665,0.450274
1382,1382,42,2.94521,0.86528,3,4,3,0.062708,699.675,43.875,...,NaN,16.430575,0.000000e+00,10.09003,28.6,2.25,9.795365,28.669335,1.916665,0.450274
1383,1383,42,2.94521,0.86528,3,4,3,0.062708,699.675,43.875,...,NaN,16.430575,0.000000e+00,10.09003,28.6,2.25,9.795365,28.669335,1.916665,0.450274
1384,1384,42,2.94521,0.86528,3,4,3,0.062708,699.675,43.875,...,NaN,16.430575,0.000000e+00,10.09003,28.6,2.25,9.795365,28.669335,1.916665,0.450274
1385,1385,42,2.94521,0.86528,3,4,3,0.062708,699.675,43.875,...,NaN,16.430575,0.000000e+00,10.09003,28.6,2.25,9.795365,28.669335,1.916665,0.450274
1386,1386,42,2.94521,0.86528,3,4,3,0.062708,699.675,43.875,...,NaN,16.430575,0.000000e+00,10.09003,28.6,2.25,9.795365,28.669335,1.916665,0.450274


In [10]:

for node in a.node.unique(): 
    temp = df_node.loc[df_node['node'] == node]['line_id'].to_numpy()
# if a node has 3 or more connections, then there can be an angle detected

[136] 223.0
[] nan
[53] 313.0
[ 53 136 176] 156.0
[176] 108.0


In [ ]:
# first single out a network: 
max_line_ids = cell_df.edges.max() 

for cc in cell_df.cc_x.unique(): 
    a = cell_df[cell_df['cc_x'] == 22]
    # find the number of line_id's in the things
    for line_id in a.line_id.unique(): 
        # find the nodes of that line_id: 
        nodes_in_df = df_node.loc[df['line_id'] == line_id]['node']


In [ ]:
a = network_df[['cc_x', 'line_id', 'x_x', 'y_x', 'z_x', 'node']]

for cc in cell_df.cc_x.unique(): 
    cc_df = a[a['cc_x'] == cc]
    for line in cc_df.line_id.unique(): 
        line_df = cc_df[cc_df['line_id'] == line]
        if len(netList) < 2: 
            netList.append(line_df)

minx  = a['x_x'].min()
maxx  = a['x_x'].max()
miny  = a['y_x'].min()
maxy  = a['y_x'].max()
minz  = a['z_x'].min()
maxz  = a['z_x'].max()
center = np.array([minx + (maxx - minx)/2, miny + (maxy - miny)/2, minz + (maxz - minz)/2])



In [ ]:
# loop through the lineids and find the center of each line id. Add it to the dataframe
for line_id in network_df['line_id'].unique():
    line_df = network_df[network_df['line_id'] == line_id]
    minx  = line_df['x_x'].min()
    maxx  = line_df['x_x'].max()
    miny  = line_df['y_x'].min()
    maxy  = line_df['y_x'].max()
    minz  = line_df['z_x'].min()
    maxz  = line_df['z_x'].max()
    center = np.array([minx + (maxx - minx)/2, miny + (maxy - miny)/2, minz + (maxz - minz)/2])
    network_df.loc[network_df['line_id'] == line_id, ['center_x', 'center_y', 'center_z']] = center
# find the distance from the center of the network to the center of each line id
